# From FORTRAN to Python

## Classes and objects

In [1]:
from copy import copy

import numpy as np

In this exercise, we are going to implement a class to represent a point in 3D space. A natural approach to this is to ask for the cartesian coordinates of the point and store them. There is another requirement however: we would like the class to show us the position in both cartesian and cylindrical coordinates. Following is a simple implementation of this concept:

In [2]:
class Point3D:
    def __init__(self, x, y, z):
        self.cart = np.array([x, y, z])
        r = np.sqrt(x**2 + y**2)
        theta = np.arctan2(y, x)
        self.cyl = np.array([r, theta, z])
        
p = Point3D(1., 1., 3.)
print(p.cart)
print(p.cyl)

[1. 1. 3.]
[1.41421356 0.78539816 3.        ]


Nice! We define the point using cartesian coordinates, and we get the cylidrical coordinates for free!
Can we also edit them?

In [8]:
p.cart[0] += 1
print(p.cart)
print(p.cyl)

[8. 1. 3.]
[1.41421356 0.78539816 3.        ]


Ok good, `p.cart` gets updated. But, `p.cyl` didn't change. What happened?

### Explanation

The `__init__` method is called the *constructor* of the Point3D class. This method is called exactly once: when the object is first created, *i.e.* when the class is *instantiated*. This means that `cyl` is only computed once from `cart`. If `cart` changes at a later point (or equivalently if `cyl` changes), it will no longer be synchronized with `cart` (resp. `cyl`.)

### Forcing coherence in classes

Objects (classes) are meant first and foremost to *hold data*. A class must represent a chunk of data to its user, and is responsible for staying coherent over time, **whatever order the user chooses to call the methods in**. In order to force coherence, object-oriented languages often use *private* variables, which the user shouldn't / can't access. In Python, nothing is ever **truly** private, but a convention is to add `_` before a private attribute or method. A safe Point3D class could look like this:

In [4]:
class Point3D_V2:
    def __init__(self, x, y, z):
        self._cart = np.array([x, y, z])
        
    def get_cart(self):
        return copy(self._cart)
    
    def get_cyl(self):
        x, y, z = self._cart
        r = np.sqrt(x**2 + y**2)
        theta = np.arctan2(y, x)
        return np.array([r, theta, z])
    
    def set_cart(self, x, y, z):
        self._cart = np.array([x, y, z])
    
p = Point3D_V2(1., 1., 3.)
print(p.get_cart())
print(p.get_cyl())

x, y, z = p.get_cart()
x += 2
p.set_cart(x, y, z)
print(p.get_cart())
print(p.get_cyl())

[1. 1. 3.]
[1.41421356 0.78539816 3.        ]
[3. 1. 3.]
[3.16227766 0.32175055 3.        ]


It works! What we have done here is effectively get rid of *redundancies* in the data. The point exists at a single location, whatever the coordinates. Even if you need to chose one representation inside the class, that does not concern the user of that class. The fact that the user can have access to several representations of the data shouldn't be confused with the need for the data to not be needlessly duplicated.

### From Java to Python

The above code works, and it is safe. It is very close to the kind of code that is usually written in Java. However, it is not considered 'Pythonic'.

  - It is more verbose than the naïve implementation of Point3D.
  - The user has to learn a lot of method names for each class, and spends time going back to the documentation.
  - You have to give enough functions to cover all cases. What if the user wants to rotate the point? Or increase the radius? Or convert to spherical coordinates? Pretty soon, you class reaches dozens and dozens of methods.
  
In an ideal world, we would like to use an interaction like for Point3D, except that it would stay coherent. This is what Python strives to offer:

In [11]:
class Point3D_V3:
    def __init__(self, x, y, z):
        self.cart = np.array([x, y, z])
    
    @property
    def cyl(self):
        x, y, z = self.cart
        r = float(np.sqrt(x**2 + y**2))
        theta = float(np.arctan2(y, x))
        return np.array([r, theta, z])

p = Point3D_V3(1., 1., 3.)
print(p.cart)
print(p.cyl)
p.cart[0] += 2
print(p.cart)
print(p.cyl)

[1. 1. 3.]
[1.41421356 0.78539816 3.        ]
[3. 1. 3.]
[3.16227766 0.32175055 3.        ]


### What just happened?

Python has a lot of tricks up its sleeve, and most of them can help you build seamless interfaces for your code. In this case, [`@property`](https://docs.python.org/3/library/functions.html#property) is a special kind of [*decorator*](https://realpython.com/primer-on-python-decorators). It "hijacks" the `cyl` method, and exposes its result as an attribute. You can't call the `cyl` method anymore, since `p.cyl` can only reference one thing, and that is now `np.array([r, theta, z])`.

In [19]:
p.cyl(0)

TypeError: 'numpy.ndarray' object is not callable

## Conclusion

The end result here is a `Point3D_V3` class that behaves in a way that is very familiar to the user. Remember the implementation of `Point3D` above? It seemed very natural, but of course as we saw it was broken. Nevertheless, because it is natural, users will spend very little time understanding and remembering it. The point of `Point3D_V3` is to keep this usability, but use a bit of dark magic (actually, [*syntactic sugar*](https://en.wikipedia.org/wiki/Syntactic_sugar)) to make it work in the background.